# Interacting with Large Language Models using Semantic Kernel

This notebook demonstrates how to interact with large language models (LLMs) using the Microsoft Semantic Kernel in .NET..

**Objectives:**
- Understand how to set up Semantic Kernel for LLM interaction in .NET.
- Learn to configure and connect to different model providers (OpenAI, Azure OpenAI, GitHub models).
- Use model parameters to customize model behavior.
- Send prompts to LLMs and receive responses.
- Use prompt templates and kernel arguments for dynamic, reusable prompts.

## Setup

In this section, we will set up the Semantic Kernel environment and configure it to use different LLM providers.

**Step 1**: Install NuGet packages

To get started with Semantic Kernel, you need to install the required NuGet packages. These packages provide the core functionality for interacting with AI models and managing environment variables. Specifically:
- `Microsoft.SemanticKernel` enables you to build and run AI-powered workflows.
- `DotNetEnv` allows you to load environment variables from a `.env` file, making it easier to manage secrets and configuration settings.

In [5]:
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 1.55.0"
#r "nuget: DotNetEnv, 3.1.0"

Installed Packages DotNetEnv, 3.1.0 Microsoft.SemanticKernel, 1.55.0

**Step 2**: Read environment variables

  In this step, we load these variables from a `.env` file (if present) so that they can be accessed by the application.


In [6]:
using DotNetEnv;
using System.IO;

var envFilePath = Path.Combine(Environment.CurrentDirectory, "../..", ".env");
if (File.Exists(envFilePath))
{
    Env.Load(envFilePath);
    Console.WriteLine($"Loaded environment variables from {envFilePath}");
}
else
{
    Console.WriteLine($"No .env file found at {envFilePath}");
}

Loaded environment variables from e:\profile_dev\aiagent-workshop\notebooks\semantic-kernel\../..\.env


**Step 3**: Instantiate the Kernel

The Semantic Kernel is the core component that orchestrates AI services and plugins. In this step, we create and configure a Kernel instance, which will be used to interact with AI models.

In [7]:
using System.ClientModel;
using OpenAI;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using System.Text;

OpenAIClient client = null;
if(Environment.GetEnvironmentVariable("USE_AZURE_OPENAI") == "true")
{
    // Configure Azure OpenAI client
    var azureEndpoint = Environment.GetEnvironmentVariable("AZURE_OPENAI_ENDPOINT");
    var apiKey = Environment.GetEnvironmentVariable("AZURE_OPENAI_API_KEY");
    client = new OpenAIClient(new ApiKeyCredential(apiKey), new OpenAIClientOptions { Endpoint = new Uri(azureEndpoint) });
}
else if(Environment.GetEnvironmentVariable("USE_OPENAI") == "true")
{
    // Configure OpenAI client
    var apiKey = Environment.GetEnvironmentVariable("OPENAI_API_KEY");
    client = new OpenAIClient(new ApiKeyCredential(apiKey));
}
else if(Environment.GetEnvironmentVariable("USE_GITHUB") == "true")
{
    // Configure GitHub model client
    var uri = Environment.GetEnvironmentVariable("GITHUB_MODEL_ENDPOINT");
    var apiKey = Environment.GetEnvironmentVariable("GITHUB_TOKEN");
    client = new OpenAIClient(new ApiKeyCredential(apiKey), new OpenAIClientOptions { Endpoint = new Uri(uri) });
}

var modelId = Environment.GetEnvironmentVariable("MODEL");
// Create a chat completion service
var builder = Kernel.CreateBuilder();
builder.AddOpenAIChatCompletion(modelId, client);

// Get the chat completion service
Kernel kernel = builder.Build();

## Calling LLMs

This section demonstrates how to call different LLMs using the Semantic Kernel.

**Step 1**: Call the Kernel

In this step, we send a prompt to the Semantic Kernel and receive a response from the AI model. 

In [19]:
using Microsoft.SemanticKernel.Connectors.OpenAI;

//call the kernel to get a response
var prompt = "What is the capital of France?";
var response = await kernel.InvokePromptAsync(prompt);
Console.WriteLine($"Response: {response}");

Response: The capital of France is **Paris**.


**Step 2**: Use Prompt Templates and Kernel Arguments

Prompt templates allow you to create reusable prompts with placeholders for dynamic values. Kernel arguments let you pass values to these placeholders at runtime, making your prompts flexible and powerful.

In [20]:
// Define a prompt template with a placeholder
string template = "What is the capital of {{$country}}?";

// Create a function from the prompt template
var promptTemplateConfig = new PromptTemplateConfig(template);
var promptTemplateFactory = new KernelPromptTemplateFactory();
var promptTemplate = promptTemplateFactory.Create(promptTemplateConfig);
var capitalFunction = kernel.CreateFunctionFromPrompt(template);

// Prepare kernel arguments
var arguments = new KernelArguments
{
    ["country"] = "Japan"
};

// Call the kernel with the function and arguments
var capitalResponse = await kernel.InvokeAsync(capitalFunction, arguments);
Console.WriteLine($"Response: {capitalResponse}");

Response: The capital of Japan is **Tokyo**.


**Step 3**: Try with another country

You can reuse the same prompt template and function with different arguments to get answers for other countries.

In [21]:
arguments["country"] = "Brazil";
capitalResponse = await kernel.InvokeAsync(capitalFunction, arguments);
Console.WriteLine($"Response: {capitalResponse}");

Response: The capital of Brazil is **Brasília**. It became the capital on April 21, 1960, replacing Rio de Janeiro. Brasília was specifically designed and built to serve as the country's capital, with its urban planning led by architect **Lúcio Costa** and the iconic buildings designed by architect **Oscar Niemeyer**.


## Model Parameters

Large language models expose several parameters that control the behavior and quality of their responses. 

**Common Model Parameters:**
- **Temperature:** Controls the randomness of the output. Lower values (e.g., 0.2) make the output more focused and deterministic, while higher values (e.g., 0.8) make it more creative and random.
- **MaxTokens:** Limits the maximum number of tokens (words or word pieces) in the response. Useful for controlling the length of the output.
- **TopP:** Makes the model pick from the most likely words until their total chance adds up to TopP (like 0.5). Lower values make the answer more focused.

> **Note:** The exact set of parameters and their effects may vary depending on the model provider (OpenAI, Azure OpenAI, etc.).

In [22]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
 
#pragma warning disable SKEXP0001
var openAiPromptSettings = new OpenAIPromptExecutionSettings
{
    MaxTokens = 100,
    Temperature = 0.5,
    TopP = 0.8
};
#pragma warning restore SKEXP0001

var chat = kernel.GetRequiredService<IChatCompletionService>(); 
// Create a prompt that will trigger the function call
string prompt = "Write a short poem about the ocean.";
var response = await chat.GetChatMessageContentAsync("Write a short poem about the ocean.", openAiPromptSettings, kernel);
Console.WriteLine($"Response: {response}");


Response: Beneath the sky so vast and wide,  
The ocean whispers, pulls, and glides.  
A mirror blue, a tempest gray,  
It sings of life and sweeps decay.  

Its waves embrace the moonlit shore,  
A timeless dance, forevermore.  
Secrets deep in shadows lie,  
Where mysteries drift and echoes sigh.  

The ocean breathes, a living soul,  
Its endless depths a story whole.  
A world of wonder, fierce and free,


You can adjust these parameters to experiment with different response styles and lengths. Try changing the temperature or max tokens and observe how the model's output changes.

## Using System Prompt and User Prompt

Semantic Kernel allows you to provide both a system prompt (to guide the model's behavior) and a user prompt (the actual user input). This is useful for customizing the assistant's persona or instructions for a conversation.

In [23]:

using Microsoft.SemanticKernel.Connectors.OpenAI;

// Improved Example: Using a detailed system prompt and a user prompt
var systemPrompt = @"You are a helpful assistant that talks like a pirate."; 

var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemPrompt);
chatHistory.AddUserMessage("Hi, can you help me?");
chatHistory.AddAssistantMessage("Arrr! Of course, me hearty! What can I do for ye?");
chatHistory.AddUserMessage("What's the best way to train a parrot?");

var chat = kernel.GetRequiredService<IChatCompletionService>(); 
var chatResponse = await chat.GetChatMessageContentAsync(chatHistory);
Console.WriteLine($"Response: {chatResponse}");

Response: Yo ho ho! Trainin’ a parrot, ye say? A fine pursuit for any pirate—or landlubber—lookin’ for a talkative feathered matey! Here's a treasure map to success:

1. **Build Trust First, Matey**: Spend time with yer parrot so it gets comfy with ye. Talk gentle-like an' offer treats or snacks—bits o' fruit or seeds do the trick.

2. **Teach with Patience, Aye**: Parrots be clever birds, but they’ve got their own rhythm, like the ebb o’ the tide. Start with simple phrases like "Ahoy!" or "Polly want a cracker?" Say it often and clear-like, and reward 'em for tryin'.

3. **Keep Sessions Short, Ye Scallywag**: Don’t tire the poor bird out! Ten to fifteen minutes a day is plenty. Ye don’t want yer parrot flyin’ off ‘cause it’s bored of yer jabberin’.

4. **Use Treats and Praise**: Every time yer parrot mimics yer words or does what ye want, give 'em a tasty morsel or a scratch under the beak. Positive reinforcement be the way o’ the pirate!

5. **Consistency is Key, Aye Aye!** Always us